# Analyzing Hubway Trips in SQL (A Tutorial Notebook)

Fetching the database from URL

In [2]:
!wget 'https://dataquest.io/blog/large_files/hubway.db'

--2022-10-03 10:43:31--  https://dataquest.io/blog/large_files/hubway.db
Resolving dataquest.io (dataquest.io)... 172.66.40.152, 172.66.43.104, 2606:4700:3108::ac42:2b68, ...
Connecting to dataquest.io (dataquest.io)|172.66.40.152|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3.amazonaws.com/dq-blog-files/hubway.db [following]
--2022-10-03 10:43:32--  https://s3.amazonaws.com/dq-blog-files/hubway.db
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.74.70
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.74.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136572928 (130M) [application/x-www-form-urlencoded]
Saving to: ‘hubway.db.3’

hubway.db.3         100%[===================>] 130.25M  97.9MB/s    in 1.3s    

2022-10-03 10:43:33 (97.9 MB/s) - ‘hubway.db.3’ saved [136572928/136572928]



### This is used to call the magic sql Library

In [3]:
%reload_ext sql
%sql sqlite:///hubway.db
%config SqlMagic.autocommit=False

**Please write your queries below in the empty boxes same as you write in a SQL Window**

Just append **%sql** before your query like this '%sql Select * from Trips limit 5;'

In [4]:
%sql Select * from trips limit 2

 * sqlite:///hubway.db
Done.


id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male


**About the Dataset and Schema**

The database has two tables, trips and stations. To begin with, we'll look at the trips table. It contains the following columns:


**Trips:**

id — A unique integer that serves as a reference for each trip

duration — The duration of the trip, measured in seconds

start_date — The date and time the trip began

start_station — An integer that corresponds to the id column in the stations table for the station the trip started at

end_date — The date and time the trip ended

end_station — The 'id' of the station the trip ended at

bike_number — Hubway's unique identifier for the bike used on the trip

sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership

zip_code — The zip code of the user (only available for registered members)

birth_date — The birth year of the user (only available for registered members)

gender — The gender of the user (only available for registered members)


**Station:**


id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)

station — The station name

municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)

lat — The latitude of the station

lng — The longitude of the station

## Analytical Questions Test in SQL

Q1. What was the duration of the longest trip? What was the average trip duration? What was the smallest trip duration?

In [5]:
%sql select MAX(duration), AVG(duration), MIN(duration) from Trips 

 * sqlite:///hubway.db
Done.


MAX(duration),AVG(duration),MIN(duration)
9999,912.4096819046612,0


Q2. How many trips were taken by 'Registered' users?

Hint: Use Sub Type to Filter

In [6]:
%sql select count(sub_type) from Trips where sub_type = 'Registered'

 * sqlite:///hubway.db
Done.


count(sub_type)
1105192


Q3. How many trips were taken by male users in comparison to Female users? for Registered users only

Hint: Group By Gender and Calculate Count

In [7]:
%sql select gender, count(sub_type = 'Registered') from Trips  group by gender

 * sqlite:///hubway.db
Done.


gender,count(sub_type = 'Registered')
,464809
Female,271333
Female,1
Male,833858


Q4. Do registered or casual users take longer trips? 

Hint: Group By Sub Type and Calculate Average

In [8]:
%sql select sub_type, avg(duration) from Trips group by sub_type

 * sqlite:///hubway.db
Done.


sub_type,avg(duration)
Casual,1519.6438967403815
Registered,657.0260669639302


Q5. Which bike was used for the most trips? 

Hint : Use Bike Number and Limit Results to 1

In [9]:
%sql select duration, MAX(bike_number) from Trips group by bike_number LIMIT 1

 * sqlite:///hubway.db
Done.


duration,MAX(bike_number)
1946,


Q6. What is the average duration of trips by users over the age of 30? 

Hint : DoB is already in years and use Airthmetic operator to caluculate Age like (2020-DOB)

In [10]:
%sql select count(birth_date), avg(duration)  from Trips  where  2020 - birth_date > 30

 * sqlite:///hubway.db
Done.


count(birth_date),avg(duration)
1549678,916.5142035958438


Q7. Which stations are most frequently used for round trips? 

Hint: You will require to use Join Station Table, Also a Round trip is considered as where Start and Stop Stations are same. Limit to top 5 Stations

In [49]:
%sql select stations.station, count(*) as "count" from trips inner join stations on trips.start_station = stations.id where trips.start_station = trips.end_station group by stations.station order by count(*) desc limit 5;


 * sqlite:///hubway.db
Done.


station,count
The Esplanade - Beacon St. at Arlington St.,3064
Charles Circle - Charles St. at Cambridge St.,2739
Boston Public Library - 700 Boylston St.,2548
Boylston St. at Arlington St.,2163
Beacon St / Mass Ave,2144


Q8. How many trips start and end in different municipalities?

Hint: We need to JOIN the trips table to the stations table twice. Once ON the start_station column and then ON the end_station

In [53]:
%sql select count(trips.id) as count from trips inner join stations as start on trips.start_station = start.id inner join stations as end  on trips.end_station = end.id where start.municipality <> end.municipality

 * sqlite:///hubway.db
Done.


count
309748


Q9. How many trips incurred additional fees (lasted longer than 30 minutes)?

Hint: Use an Arithmetic function to calculate Duration in Minutes and Use in Where Clause

In [76]:
%sql select count(*) as Count_of_Duration_longer_than_30mins from trips where duration > (30*60)

 * sqlite:///hubway.db
Done.


Count_of_Duration_longer_than_30mins
123155


Q10. Which bike was used for the longest total time? Provide the answer in Hours

Hint: Use an Group by Bike Number to sum Duration in Seconds and Limit Result to 1

In [86]:
%sql select bike_number, sum(duration)/3600 as total_duration from trips group by bike_number order by total_duration desc limit 1

 * sqlite:///hubway.db
Done.


bike_number,total_duration
B00490,571


Q11. Did registered or casual users take more round trips?

Hint: Recall Q7 and Modify your query to answer by Sub Type

In [87]:
%sql select sub_type, count(*) as "count" from trips inner join stations on trips.start_station = stations.id where trips.start_station = trips.end_station group by sub_type order by count(*) desc limit 5;


 * sqlite:///hubway.db
Done.


sub_type,count
Casual,41427
Registered,31635


Q12. Which municipality had the most frequent Station End?

Hint: Join the Station table on End Station and then Group by Municipality to find answer

In [90]:
%sql select stations.municipality, avg(duration) as AVG from trips inner join stations on trips.start_station = stations.id where trips.end_station = stations.id group by Municipality order by count(*) desc limit 5;


 * sqlite:///hubway.db
Done.


municipality,AVG
Boston,2216.5568143728633
Cambridge,1561.677794448612
Somerville,1921.036111111111
Brookline,1895.7236559139785
